In [1]:
from src.envs.smoothenv.Pendulum import InvertedPendulum
from src.dyn_model.env1.env1 import InvertedPendulum as InvertedPendulum1
import jax 
import jax.numpy as jnp
import orbax.checkpoint as ocp
import os 

In [2]:
env = InvertedPendulum()

11


In [3]:
env1 = InvertedPendulum1()
rng1 = jax.random.PRNGKey(100)
state = env1.reset(rng1)
actions = jnp.ones((100,1))
states = jnp.ones((100,4))
for i in range(100):
    actions = actions.at[i].set(jax.random.normal(jax.random.PRNGKey(i), (1,)))

for i in range(100):
    state = env1.step(state,actions[i])
    q,qd = state.pipeline_state.q,state.pipeline_state.qd
    qqd = jnp.concatenate((q,qd))
    states = states.at[i].set(qqd)



11


In [4]:
rng = jax.random.PRNGKey(0)
state = env.reset(rng)
ckptr = ocp.AsyncCheckpointer(ocp.StandardCheckpointHandler())
ckpt_dir = os.path.abspath('src/dyn_model/params/1')
params = ckptr.restore(ckpt_dir)
state = env.step(state=state, action=jnp.array([0.0]),params=params)


params_updated = env.tunemodel(params,states,actions)

print("params:",params)
print("updated parameters:",params_updated)


params: {'dense1': {'bias': Array([ 1.7094061 , -0.26725167,  2.0695124 ,  0.60761   ,  1.3507779 ,
       -0.7891483 ,  0.53850013, -0.12914258], dtype=float32), 'kernel': Array([[-0.04097296, -0.19262417, -0.30121544,  0.17111811, -0.19610043,
        -0.18503188,  0.4814186 ,  0.0119962 ],
       [ 0.72658515,  0.04036251,  0.09475983,  0.22971356, -0.32784072,
         0.13116011, -0.1360797 ,  0.06588838],
       [ 0.41956756, -0.04538892,  1.0171367 ,  0.42176297,  0.49802816,
        -0.17976578,  0.18255067, -0.54494584],
       [-0.16771728,  0.15369727,  0.6447564 ,  0.20203042, -0.38094702,
         1.0313072 , -0.49459273, -0.62510777],
       [ 0.12005295, -0.00889426, -0.21451057,  0.31970394,  0.35107115,
        -0.39675918,  0.04405806,  0.03365863]], dtype=float32)}, 'dense2': {'bias': Array([ 0.04861681,  0.07943715,  0.09077338,  0.05503768,  0.23357557,
        0.93783253,  0.7151339 ,  0.14828236,  0.11467941,  0.01748987,
        0.48013905, -0.36506057,  0.54899